## Ассоциативные правила

### Задание 
Исследовать данные на наличие ассоциативных правил алгоритмом Apriori. Подобрать значения Supp и Conf. Интерпретировать полученные результаты.

### Данные
* DataSet9_1 – Данные о статистике продаж
* DataSet9_2 – Статистика продаж легковых автомобилей в США

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori

In [46]:
data = pd.read_csv('C:/Users/rosti/Desktop/Преподавание/ВШЭ/ML/Практики/R/HSE ML/DataSets/DataSet9_2.csv', sep = ',', on_bad_lines='skip')

In [3]:
data.head()

,Id,Price,Year,Mileage,City,State,Vin,Make,Model
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


### Обучение ассоциативным правилам
Обучение ассоциативным правилам  —  одна из важнейших концепций машинного обучения. Применяется в анализе рыночной корзины и статистики посещения сайтов, непрерывном производстве и т. д. Анализ рыночной корзины  —  это метод, используемый в крупных розничных сетях для выявления ассоциативных связей между товарами. Для его понимания показателен пример супермаркета, где все покупаемые вместе продукты раскладываются на полках рядом.
### Введение в APRIORI
В основе Apriori  —  поиск частотных множеств элементов в наборе данных. Этот алгоритм построен на ассоциациях и корреляциях между наборами элементов. Он применяется на рекомендательных платформах  —  там, где мы обычно видим «вам также может понравиться».<br />
В алгоритме Apriori предполагается, что любое подмножество частотного набора элементов должно быть частотным. Например, если транзакция {молоко, яйца, хлеб} частотна, должна быть частотной и ее составляющая {яйца, хлеб}.
### Принцип работы Apriori
Чтобы из всего многообразия правил отобрать интересные, для примера супермаркета применим следующие показатели:
* Support (поддержка) - в общем виде это показатель «частотности»; 
* Lift (лифт) - это отношение «зависимости» items к их «независимости». Lift показывает, насколько items зависят друг от друга;
* Confidence (уверенность) - как часто наше правило срабатывает для всего датасета.

In [4]:
#Библиотека APYORI требует использование списка списков.

data = data.astype(str)
data['Make + Model'] = data['Make'] + ' ' + data['Model']
data['City + State'] = data['City'] + ' ' + data['State']
data.drop(['Make', 'Model', 'Vin', 'City', 'State', 'Id'], axis=1, inplace = True) 
transactions = data.values.tolist()
data.head()

,Price,Year,Mileage,Make + Model,City + State
0,16472,2015,18681,Buick EncoreConvenience,Jefferson City MO
1,15749,2015,27592,Buick EncoreFWD,Highland IN
2,16998,2015,13650,Buick EncoreLeather,Boone NC
3,15777,2015,25195,Buick EncoreFWD,New Orleans LA
4,16784,2015,22800,Buick EncoreConvenience,Las Vegas NV


In [18]:
rules = apriori(transactions = transactions, min_support = 0.001, min_cinfidence = 0.001, min_lift = 1.1, min_length = 2, max_length = 2)
results = list(rules)

In [29]:
def inspect(results):
    lhs         =[tuple(result[2][0][0])[0] for result in results]
    rhs         =[tuple(result[2][0][1])[0] for result in results]
    supports    =[result[1] for result in results]
    confidences =[result[2][0][2] for result in results]
    lifts        =[result[2][0][3] for result in results]
    return list (zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ["Left hand side", "Right hand side", "Support", "Confidence", "Lift"])

In [37]:
resultsinDataFrame = resultsinDataFrame.nlargest(n = 10, columns = "Lift")

In [38]:
resultsinDataFrame

,Left hand side,Right hand side,Support,Confidence,Lift
5,2013,Nissan AltimaSedan,0.001399,0.016331,11.669913
2,2013,Ford F-150SuperCrew,0.001972,0.023018,11.660192
118,2017,Ram 1500SLT,0.001306,0.011164,8.529265
117,2017,Mercedes-Benz C-ClassC,0.001482,0.012676,8.378805
102,2017,Chevrolet MalibuLT,0.005041,0.043104,7.271102
1,2011,Hyundai Sonata4dr,0.001034,0.021953,6.763032
101,2017,Chevrolet CruzeSedan,0.002917,0.024944,6.737299
67,2016,Chevrolet Cruze,0.002089,0.013437,6.431578
68,2016,Chevrolet Malibu,0.001386,0.008916,6.047764
115,2017,Kia SedonaLX,0.001329,0.011361,5.798578


In [41]:
data_10_best = resultsinDataFrame[:10]

In [42]:
import plotly.express as px
fig = px.scatter(data_10_best, x='Support', y='Confidence', color='Lift')
fig.show()

In [43]:
import plotly.express as px
fig = px.scatter(data_10_best, x='Left hand side', y='Right hand side', color='Lift', size = 'Support')
fig.show()